## Human-in-the-Loop
There are two main ways to interact with the team from your application:

1. During a team’s run – execution of run() or run_stream(), provide feedback through a UserProxyAgent.

2. Once the run terminates, provide feedback through input to the next call to run() or run_stream().

## Load Azure Configurations

In [1]:
from dotenv import load_dotenv
import os

azure_openai_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
azure_openai_key = os.getenv("AZURE_OPENAI_API_KEY")
azure_openai_deployment = os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME")
azure_openai_api_version = os.getenv("AZURE_OPENAI_API_VERSION")

## Create Azure OpenAI Client
Using the model client class

In [2]:
from autogen_ext.models.openai import AzureOpenAIChatCompletionClient
from azure.identity import DefaultAzureCredential, get_bearer_token_provider

# Create the token provider
#token_provider = get_bearer_token_provider(DefaultAzureCredential(), "https://cognitiveservices.azure.com/.default")

az_model_client = AzureOpenAIChatCompletionClient(
    azure_deployment=azure_openai_deployment,
    model=azure_openai_deployment,
    api_version=azure_openai_api_version,
    azure_endpoint=azure_openai_endpoint,
    # azure_ad_token_provider=token_provider,  # Optional if you choose key-based authentication.
    api_key=azure_openai_key, # For key-based authentication.
)

## UserProxyAgent - Providing Feedback During a Run
The UserProxyAgent is a special built-in agent that acts as a proxy for a user to provide feedback to the team.

To use the UserProxyAgent, you can create an instance of it and include it in the team before running the team. The team will decide when to call the UserProxyAgent to ask for feedback from the user.

When UserProxyAgent is called during a run, it blocks the execution of the team until the user provides feedback or errors out. This will hold up the team’s progress and put the team in an unstable state that cannot be saved or resumed.

Due to the blocking nature of this approach, it is recommended to use it only for short interactions that require immediate feedback from the user, such as asking for approval or disapproval with a button click, or an alert requiring immediate attention otherwise failing the task.

In [3]:
from autogen_agentchat.agents import AssistantAgent, UserProxyAgent
from autogen_agentchat.conditions import TextMentionTermination
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_agentchat.ui import Console

# Create the agents.
# Create the primary agent.
primary_agent = AssistantAgent(
    "primary",
    model_client=az_model_client,
    system_message="You are a helpful AI assistant.",
)

# Create the critic agent.
critic_agent = AssistantAgent(
    "critic",
    model_client=az_model_client,
    system_message="Provide constructive feedback.",
)

# Create the user proxy agent.
user_proxy = UserProxyAgent("user_proxy", input_func=input)  # Use input() to get user input from console.

# Create the termination condition which will end the conversation when the user says "APPROVE".
termination = TextMentionTermination("APPROVE")

# Create the team.
team = RoundRobinGroupChat([primary_agent, critic_agent, user_proxy], termination_condition=termination)

# Run the conversation and stream to the console.
stream = team.run_stream(task="Write a short poem about the fall season.")
# Use asyncio.run(...) when running in a script.
await Console(stream)

---------- user ----------
Write a short poem about the fall season.
---------- primary ----------
Golden leaves in crisp embrace,
Whisper secrets through the breeze,
Amber skies in twilight's grace,
Paint a world that aims to please.

Pumpkins dot the fields anew,
Harvest moons cast silver beams,
Sweaters wrap in cozy hue,
Autumn stirs our sweetest dreams.

Chilly air with apple scent,
Misty mornings, soft and gray,
Nature's song is heaven-sent,
As fall adorns the fleeting day.
[Prompt tokens: 28, Completion tokens: 88]


/home/codespace/.python/current/lib/python3.12/site-packages/autogen_agentchat/agents/_assistant_agent.py:343: UserWarning: Resolved model mismatch: gpt-4o-2024-08-06 != gpt-4o-2024-05-13. Model mapping may be incorrect.
  result = await self._model_client.create(


---------- critic ----------
Your poem beautifully captures the essence of the fall season. The imagery of golden leaves, amber skies, and pumpkins in the fields creates a vivid picture of autumn’s beauty. The use of sensory details, such as the chilly air with an apple scent and misty mornings, effectively brings the reader into the scene. Your choice of words, like "whisper," "twilight's grace," and "heaven-sent," adds a poetic and lyrical quality to the piece.

However, there are a few suggestions to consider:

1. **Consistency of Rhythmic Flow**: While the poem has a lovely rhythm, maintaining a consistent meter throughout can enhance the flow. For example, the second stanza could benefit from slight adjustments to match the syllabic pattern of the first stanza more closely.

2. **Variation in Imagery**: You lean heavily on visual imagery, which is beautiful, but incorporating other senses can add depth. You've hinted at scent and touch (e.g., chilly air, sweaters), and further dev

TaskResult(messages=[TextMessage(source='user', models_usage=None, content='Write a short poem about the fall season.', type='TextMessage'), TextMessage(source='primary', models_usage=RequestUsage(prompt_tokens=28, completion_tokens=88), content="Golden leaves in crisp embrace,\nWhisper secrets through the breeze,\nAmber skies in twilight's grace,\nPaint a world that aims to please.\n\nPumpkins dot the fields anew,\nHarvest moons cast silver beams,\nSweaters wrap in cozy hue,\nAutumn stirs our sweetest dreams.\n\nChilly air with apple scent,\nMisty mornings, soft and gray,\nNature's song is heaven-sent,\nAs fall adorns the fleeting day.", type='TextMessage'), TextMessage(source='critic', models_usage=RequestUsage(prompt_tokens=118, completion_tokens=281), content='Your poem beautifully captures the essence of the fall season. The imagery of golden leaves, amber skies, and pumpkins in the fields creates a vivid picture of autumn’s beauty. The use of sensory details, such as the chilly a

## Using Max Turns

This method allows you to pause the team for user input by setting a maximum number of turns. 

To implement this, set the max_turns parameter in the RoundRobinGroupChat() constructor.

Once the team stops, the turn count will be reset. 

When you resume the team, it will start from 0 again. However, the team’s internal state will be preserved, for example, the RoundRobinGroupChat will resume from the next agent in the list with the same conversation history.

In [ ]:
# Create the team setting a maximum number of turns to 4.
# One chat by agent is one turn. 
team = RoundRobinGroupChat([primary_agent, critic_agent], max_turns=4)

task = "Write a short poem about the fall season."
while True:
    # Run the conversation and stream to the console.
    stream = team.run_stream(task=task)
    # Use asyncio.run(...) when running in a script.
    await Console(stream)
    # Get the user response.
    task = input("Enter your feedback (type 'exit' to leave): ")
    if task.lower().strip() == "exit":
        break

---------- user ----------
Write a short poem about the fall season.


---------- primary ----------
Autumn drapes in colors bold,
Russet, amber, red, and gold,
Whispers through the rustling trees,
Chilled embraces from the breeze.

Pumpkins line the farmer's field,
Bounties of the earth revealed,
Cider warms in mugs so bright,
Harvest moon whispers goodnight.

Sweater-clad, we wander slow,
In the twilight's gentle glow,
Nature's lullaby, so sweet,
In fall’s embrace, our hearts meet.
[Prompt tokens: 243, Completion tokens: 97]
---------- critic ----------
Your poem is a lovely portrayal of the fall season, with its rich imagery and soothing tone. You effectively capture the visual splendor of autumn and the comforting elements that make this season special. Here are some points to consider:

1. **Imagery and Word Choice**: The use of words like "russet," "amber," and "gold" beautifully conveys the colors of fall. Describing the visual elements is well done, and the tactile sensations (like the chilled breeze) add a nice touch.

2. **Rhyme Scheme**: The co

## HandoffTermination 
HandoffTermination stops the team when an agent sends a HandoffMessage message.

Note: The model used with AssistantAgent must support tool call to use the handoff feature.

In [6]:
from autogen_agentchat.base import Handoff
from autogen_agentchat.conditions import HandoffTermination

# Create the agents.
# Create the primary agent.
primary_agent = AssistantAgent(
    "primary",
    model_client=az_model_client,
    system_message="You are a helpful AI assistant.",
)

# Create the critic agent.
critic_agent = AssistantAgent(
    "critic",
    model_client=az_model_client,
    system_message="Provide constructive feedback first then transfer to user.",
    handoffs=[Handoff(target="user", message="Transfer to user.")],
)

# Define a termination condition that checks for handoff message targetting helper OR text "APPROVE".
handoff_termination = HandoffTermination(target="user")

# Create the team.
team = RoundRobinGroupChat([primary_agent, critic_agent], termination_condition=handoff_termination)

# Run the conversation and stream to the console.
stream = team.run_stream(task="Write a short poem about the fall season.")
# Use asyncio.run(...) when running in a script.
await Console(stream)


---------- user ----------
Write a short poem about the fall season.
---------- primary ----------
Golden leaves in whispered fall,
Summer's warmth starts to withdraw.
Crisp air dances, soft and light,
As days blend gently into night.

Pumpkin hues and apples sweet,
Nature's bounty at our feet.
Sweater weather, cozy hours,
Autumn's magic, subtle power.

Against the sky, in twilight's glow,
Geese in formation southward flow.
With every breeze, a story's cast,
Of seasons turning, time amassed.

Embrace this change, a fleeting call,
The splendid wonder known as fall.
[Prompt tokens: 28, Completion tokens: 109]
---------- critic ----------
Your poem beautifully captures the essence of fall, painting vivid pictures through your words. Here are a few aspects that particularly stand out:

1. **Imagery and Descriptions**: You've used evocative language that brings the fall season to life. Phrases like "Golden leaves in whispered fall" and "Crisp air dances, soft and light" create a sensory exp

TaskResult(messages=[TextMessage(source='user', models_usage=None, content='Write a short poem about the fall season.', type='TextMessage'), TextMessage(source='primary', models_usage=RequestUsage(prompt_tokens=28, completion_tokens=109), content="Golden leaves in whispered fall,\nSummer's warmth starts to withdraw.\nCrisp air dances, soft and light,\nAs days blend gently into night.\n\nPumpkin hues and apples sweet,\nNature's bounty at our feet.\nSweater weather, cozy hours,\nAutumn's magic, subtle power.\n\nAgainst the sky, in twilight's glow,\nGeese in formation southward flow.\nWith every breeze, a story's cast,\nOf seasons turning, time amassed.\n\nEmbrace this change, a fleeting call,\nThe splendid wonder known as fall.", type='TextMessage'), TextMessage(source='critic', models_usage=RequestUsage(prompt_tokens=170, completion_tokens=234), content='Your poem beautifully captures the essence of fall, painting vivid pictures through your words. Here are a few aspects that particular